In [6]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from PIL import Image
import io
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
from __future__ import annotations

from typing import Any

from sentinelhub import SHConfig

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)

def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

CLIENT_ID = "b793056a-2cc8-4a3c-b365-1a4e219fc17a"
CLIENT_SECRET = "EzK1YOACy43LCbiZGXMuYlKvPbI44gOy"

In [4]:
current_path = os.getcwd()
print(current_path)

C:\Users\PhilipWu


In [7]:


config = SHConfig()
config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET
config.sh_url = 'https://services.sentinel-hub.com/api/v1/process'
config.sh_oauth_url = 'https://services.sentinel-hub.com/oauth2/token'

config.save()

betsiboka_coords_wgs84 = (26.16, 16.15, 26.51, 15.58)
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
time_interval = ("2020-06-01", "2020-06-30")
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=resolution)


In [11]:

evalscript_true_color = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

request_true_color = SentinelHubRequest(
    evalscript=evalscript_true_color,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C,
            time_interval=("2020-06-01", "2020-06-30"),
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=betsiboka_bbox,
    size=betsiboka_size,
    config=config,
)

true_color_response = request_true_color.get_data()

true_color_img = request_true_color.get_data(save_data=True)

image = Image.open(true_color_img)

image_data = np.array(image)

df = pd.DataFrame(image_data.reshape(-1, image_data.shape[-1] if len(image_data.shape) == 3 else 1))

print(df)


print(true_color_response)

#print(all_bands_response)

image = true_color_response[0]
plot_image(image, factor = 3.5 / 255, clip_range = (0, 1))

ValueError: Request parameter `data_folder` is not specified. In order to save data please set `data_folder` to location on your disk.

In [22]:
!pip install pandas
!pip install rasterio


In [54]:
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig,
)
from PIL import Image
import numpy as np
import pandas as pd
import os
import rasterio
from datetime import date
from dateutil.relativedelta import relativedelta


# Set your SentinelHub instance configuration (e.g., client ID, secret)
CLIENT_ID = "b793056a-2cc8-4a3c-b365-1a4e219fc17a"
CLIENT_SECRET = "EzK1YOACy43LCbiZGXMuYlKvPbI44gOy"

# Set up SentinelHub config
config = SHConfig()
config.sh_client_id = CLIENT_ID
config.sh_client_secret = CLIENT_SECRET
config.sh_url = 'https://services.sentinel-hub.com/api/v1/process'
config.sh_oauth_url = 'https://services.sentinel-hub.com/oauth2/token'

config.save()


# Define your bounding box and size (you can replace these with your values)
# Example: Bounding box for Betsiboka region in Madagascar (replace with your own)
betsiboka_coords_wgs84 = (45.0, -17.0, 45.5, -16.5)  # [min_lon, min_lat, max_lon, max_lat]
resolution = 60
betsiboka_bbox = BBox(bbox=betsiboka_coords_wgs84, crs=CRS.WGS84)
betsiboka_size = bbox_to_dimensions(betsiboka_bbox, resolution=60)  # You can set the resolution

# Define the time interval for the time series
#time_interval = ("2020-06-01", "2024-11-18")



# forming sequence of time intervals for time series data

today = date.today()

time_intervals = []

for month in range(1,13):
    start = today - relativedelta(months=month)
    end = start + relativedelta(months=1)
    start = start.strftime("%Y-%m-%d")
    end = end.strftime("%Y-%m-%d")
    time_interval = (start, end)
    time_intervals.insert(0, time_interval)

#time_interval = ("2024-06-03", "2024-11-18")
    
for time_interval in time_intervals: 
    print(time_interval)




# Define the evalscript for true color images (you can adjust it as needed)
evalscript_sentinel_2 = """
//VERSION=3
function setup() {
    return {
        input: ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
        output: { bands: 13 }
    };
}

function evaluatePixel(sample) {
    return [sample.B02,
            sample.B03,
            sample.B04,
            sample.B05,
            sample.B06,
            sample.B07,
            sample.B08,
            sample.B8A,
            sample.B11,
            sample.B12,
            (sample.B8 - sample.B4) / (sample.B8 + sample.B4),
            (sample.B3 - sample.B8) / (sample.B3 + sample.B8),
            (sample.B4 - sample.B3) / sample.B8,
            
        ];
}
"""

evalscript_sentinel_1 = """
//VERSION=3
function setup() {
    return {
        input: ["VV", "VH"],
        output: { bands: 2 }
    };
}

function evaluatePixel(sample) {
    return [sample.VV,
            sample.VH
        ];
}
"""

data_folder = "data"
os.makedirs(data_folder, exist_ok=True)




time_series_df = pd.DataFrame()

for time_interval in time_intervals: 

    # Create SentinelHub request for time series data from Sentinel-2 
    request_sentinel_2_bands = SentinelHubRequest(
        evalscript=evalscript_sentinel_2,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L1C,
                time_interval=time_interval,
                #mosaicking_order='TILE',
                mosaicking_order=MosaickingOrder.LEAST_CC
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
        data_folder=data_folder
    )

    # Create sentinel

    # Get the time series data (download the TIFF)

    sentinel_2_bands_img = request_sentinel_2_bands.get_data(save_data=True)

    # Open the TIFF image using PIL
    
    print(type(sentinel_2_bands_img))
    print( sentinel_2_bands_img)
    #image = Image.open(true_color_img)

    # Convert the image to a numpy array
    image_data = np.array(sentinel_2_bands_img)

    print("Image Data")
    print(image_data.shape)
    print(image_data)

    image_squeezed = image_data.squeeze()

    #image_flattened = image_squeezed.reshape(-1, 3)

    image_flattened = image_squeezed.reshape(-1, 13)

    # Convert the numpy array to a pandas DataFrame
    # If the image is RGB, the shape will be (height, width, 3), flatten it to (height*width, 3)
    sentinel_2_df = pd.DataFrame(image_flattened, columns=[
        f'B02{time_interval}',
        f'B03{time_interval}',
        f'B04{time_interval}',
        f'B05{time_interval}',
        f'B06{time_interval}',
        f'B07{time_interval}',
        f'B08{time_interval}',
        f'B8A{time_interval}',
        f'B11{time_interval}',
        f'B12{time_interval}',
        f'NDVI{time_interval}',
        f'NDWI{time_interval}',
        f'PSRI{time_interval}',
    ])


    # repeat process for sentinel-1 api (for bands VV and VH)

    request_sentinel_1_bands = SentinelHubRequest(
        evalscript=evalscript_sentinel_1,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL1_IW,
                time_interval=time_interval,
                #mosaicking_order='TILE',
                #mosaicking_order=MosaickingOrder.LEAST_CC
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=betsiboka_bbox,
        size=betsiboka_size,
        config=config,
        data_folder=data_folder
    )

    print(f"type returned from SentinelHubRequest {type(request_sentinel_1_bands)}")

    # Get the time series data (download the TIFF)

    sentinel_1_bands_img = request_sentinel_1_bands.get_data(save_data=True)

    # Convert the image to a numpy array
    image_data = np.array(sentinel_1_bands_img)

    print("Image Data")
    print(image_data.shape)
    print(image_data)

    image_squeezed = image_data.squeeze()

    image_flattened = image_squeezed.reshape(-1, 2)

    # Convert the numpy array to a pandas DataFrame
    # If the image is RGB, the shape will be (height, width, 3), flatten it to (height*width, 3)
    sentinel_1_df = pd.DataFrame(image_flattened, columns=[f'VV{time_interval}', f'VH{time_interval}'])

    combined_sentinel_df = pd.concat([sentinel_2_df, sentinel_1_df], axis=1)

    time_series_df = pd.concat([time_series_df, combined_sentinel_df], axis=1)
    # Display the first few rows of the DataFrame
    print(time_series_df)

print('done')

## TO DO: extract functions from this giant ass block of code

('2023-11-18', '2023-12-18')
('2023-12-18', '2024-01-18')
('2024-01-18', '2024-02-18')
('2024-02-18', '2024-03-18')
('2024-03-18', '2024-04-18')
('2024-04-18', '2024-05-18')
('2024-05-18', '2024-06-18')
('2024-06-18', '2024-07-18')
('2024-07-18', '2024-08-18')
('2024-08-18', '2024-09-18')
('2024-09-18', '2024-10-18')
('2024-10-18', '2024-11-18')
<class 'list'>
[array([[[31, 32, 47, ...,  0,  0,  0],
        [29, 27, 32, ...,  0,  0,  0],
        [30, 29, 30, ...,  0,  0,  0],
        ...,
        [26, 24, 19, ...,  0,  0,  0],
        [26, 27, 21, ...,  0,  0,  0],
        [26, 26, 20, ...,  0,  0,  0]],

       [[31, 30, 44, ...,  0,  0,  0],
        [30, 28, 35, ...,  0,  0,  0],
        [32, 31, 40, ...,  0,  0,  0],
        ...,
        [29, 28, 26, ...,  0,  0,  0],
        [26, 25, 19, ...,  0,  0,  0],
        [26, 26, 20, ...,  0,  0,  0]],

       [[31, 32, 45, ...,  0,  0,  0],
        [30, 27, 30, ...,  0,  0,  0],
        [31, 31, 37, ...,  0,  0,  0],
        ...,
        

In [66]:
### HOLDS EVALSCRIPTS

evalscript_sentinel_2 = """
//VERSION=3
function setup() {
    return {
        input: ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
        output: { bands: 13 }
    };
}

function evaluatePixel(sample) {
    return [sample.B02,
            sample.B03,
            sample.B04,
            sample.B05,
            sample.B06,
            sample.B07,
            sample.B08,
            sample.B8A,
            sample.B11,
            sample.B12,
            (sample.B08 - sample.B04) / (sample.B08 + sample.B04),
            (sample.B03 - sample.B08) / (sample.B03 + sample.B08),
            (sample.B04 - sample.B03) / sample.B08,
            
        ];
}
"""

evalscript_sentinel_1 = """
    //VERSION=3
    function setup() {
        return {
            input: ["VV", "VH"],
            output: { bands: 2 }
        };
    }

    function evaluatePixel(sample) {
        return [sample.VV,
                sample.VH
            ];
}
"""

In [67]:
### imports and util functions

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
    SHConfig,
)
from PIL import Image
import numpy as np
import pandas as pd
import os
import rasterio
from datetime import date
from dateutil.relativedelta import relativedelta


def bbox_generator(coords_wgs84, resolution=60):
    bbox = BBox(bbox=coords_wgs84, crs=CRS.WGS84)
    size = bbox_to_dimensions(bbox, resolution)  # You can set the resolution
    return bbox, size


def get_time_intervals():
    today = date.today()
    time_intervals = []
    for month in range(1,13):
        start = today - relativedelta(months=month)
        end = start + relativedelta(months=1)
        start = start.strftime("%Y-%m-%d")
        end = end.strftime("%Y-%m-%d")
        time_interval = (start, end)
        time_intervals.insert(0, time_interval)
    return time_intervals

def generate_sentinel_hub_request(bbox, size, time_interval, datafolder, evalscript, data_collection): ##considered turning this into a class
    return SentinelHubRequest(
        evalscript=evalscript,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=data_collection,
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=bbox,
        size=size,
        config=config,
        data_folder=data_folder
    )

def generate_sentinel_1_request(bbox, size, time_interval, datafolder):
    return generate_sentinel_hub_request(bbox, size, time_interval, datafolder, evalscript=evalscript_sentinel_1, data_collection=DataCollection.SENTINEL1_IW)

def generate_sentinel_2_request(bbox, size, time_interval, datafolder):
    return generate_sentinel_hub_request(bbox, size, time_interval, datafolder, evalscript=evalscript_sentinel_2, data_collection=DataCollection.SENTINEL2_L1C)

def convert_request_to_df(request, no_of_bands, columns):
    image = request.get_data(save_data=True)

    # Convert the image to a numpy array
    image_data = np.array(image)
    image_squeezed = image_data.squeeze()
    image_flattened = image_squeezed.reshape(-1, no_of_bands)

    # Convert the numpy array to a pandas DataFrame
    df = pd.DataFrame(image_flattened, columns=columns)
    return df



In [68]:
def get_time_series_df_from_bbox(bbox_coords_wgs84, resolution=60):
    bbox, size = bbox_generator(bbox_coords_wgs84, resolution)
    time_intervals = get_time_intervals()
    
    data_folder = "data"
    os.makedirs(data_folder, exist_ok=True)




    band_df_dict = []

    for _ in range(1, 16)
        band_df_dict.append(pd.DataFrame())
    
    b02_df = pd.DataFrame()
    b03_df = pd.DataFrame()
    b04_df = pd.DataFrame()
    b05_df = pd.DataFrame()
    b06_df = pd.DataFrame()
    b07_df = pd.DataFrame()
    b08_df = pd.DataFrame()
    b8a_df = pd.DataFrame()
    b11_df = pd.DataFrame()
    b12_df = pd.DataFrame()
    ndvi_df = pd.DataFrame()
    ndwi_df = pd.DataFrame()
    psri_df = pd.DataFrame()
    vv_df = pd.DataFrame()
    vh_df = pd.DataFrame()

    
    time_series_df = pd.DataFrame()

    for time_interval in time_intervals: 
        sentinel_2_request = generate_sentinel_2_request(bbox, size, time_interval, data_folder)
        sentinel_2_columns = [
            f'B02{time_interval}',
            f'B03{time_interval}',
            f'B04{time_interval}',
            f'B05{time_interval}',
            f'B06{time_interval}',
            f'B07{time_interval}',
            f'B08{time_interval}',
            f'B8A{time_interval}',
            f'B11{time_interval}',
            f'B12{time_interval}',
            f'NDVI{time_interval}',
            f'NDWI{time_interval}',
            f'PSRI{time_interval}',
        ]
        sentinel_2_df = convert_request_to_df(sentinel_2_request, 13, sentinel_2_columns)

        sentinel_1_request = generate_sentinel_1_request(bbox, size, time_interval, data_folder)
        sentinel_1_columns = [
            f'VV{time_interval}', 
            f'VH{time_interval}'
        ]
        sentinel_1_df = convert_request_to_df(sentinel_1_request, 2, sentinel_1_columns)

        for index, (column_name, column_data) in enumerate(sentinel_1_df.iteritems()):
            band_df_dict[index] = concat(band_df_dict[index]
            
            
        

        combined_sentinel_df = pd.concat([sentinel_2_df, sentinel_1_df], axis=1)
        time_series_df = pd.concat([time_series_df, combined_sentinel_df], axis=1)
        print(time_series_df)

    return time_series_df
        
        
        

In [69]:
df = get_time_series_df_from_bbox((26.16, 16.15, 26.51, 15.58))

print(df)
print('done')


        B02('2023-11-18', '2023-12-18')  B03('2023-11-18', '2023-12-18')  \
0                                    37                               48   
1                                    38                               49   
2                                    39                               50   
3                                    38                               49   
4                                    38                               49   
...                                 ...                              ...   
658767                               45                               60   
658768                               47                               65   
658769                               47                               65   
658770                               50                               66   
658771                               50                               67   

        B04('2023-11-18', '2023-12-18')  B05('2023-11-18', '2023-12-18')  \
0          